# Expand it, if you want

In [ ]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

In [7]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import TensorBoard, Callback
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt
np.random.seed(123)

In [4]:
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size=3500,
                                                                        test_sample_size=400,size=28)

x_train = np.array(x_train_original)
x_test = np.array(x_test_original)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test = keras.utils.to_categorical(y_test_original, num_classes=10)

x_val = x_test[:200]
x_test = x_test[200:]
y_val = y_test[:200]
y_test = y_test[200:]

In [5]:
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.00001),
              metrics=['accuracy'])

# Write Callback

In [27]:
class AccControl(Callback):
    def __init__(self, persent=90, verbose=False):
        super(AccControl, self).__init__()
        self.persent = persent
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= 0.7:
            self.model.stop_training = True
        if self.verbose:
            val_loss = logs.get('val_loss')
            print(f'\n\n[Training Stop]   Accuracy reach 70% in epoch {epoch} with validation loss {val_loss:.3f}\n\n')
            

acc_control = AccControl(persent=70, verbose=True)
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data = (x_val, y_val), callbacks=[acc_control], verbose=1)

Epoch 1/30
108/110 [============================>.] - ETA: 0s - loss: 0.9810 - accuracy: 0.6918

[Training Stop]   Accuracy reach 70% in epoch 0 with validation loss 0.662


110/110 [==============================] - 5s 45ms/step - loss: 0.9784 - accuracy: 0.6920 - val_loss: 0.6616 - val_accuracy: 0.8950
